In [232]:
import Scrapers.ackerman_scraper as ack_scraper
import numpy as np
import matplotlib.pyplot as plt

In [167]:
raw = ack_scraper.dataframe_ackerman_auction()

{1185: '183 Apr 7, 2018 (NY)', 1187: '183A Apr 21, 2018 (HK)', 1188: '184 May 16, 2018 (NY)', 1190: '184A Jun 9, 2018 (HK)', 1192: '185 Jun 20, 2018 (NY)', 1184: '182A Mar 24, 2018 (HK)', 1196: '186 Sep 7, 2018 (NY)', 1197: '185A Sep 22, 2018 (HK)', 1165: '175A Sep 23, 2017 (HK)', 1168: '177 Oct 13, 2017 (NY)', 1202: '187 Oct 12, 2018 (NY)', 1172: '178 Nov 18, 2017 (NY)', 1173: '177A Dec 9, 2017 (HK)', 1174: '179 Dec 13, 2017 (NY)', 1177: '181A Jan 27, 2018 (HK)', 1179: '181 Feb 9, 2018 (NY)', 1163: '176 Sep 9, 2017 (NY)', 1181: '182 Mar 10, 2018 (NY)', 1170: '176A Nov 4, 2017 (HK)'}
1185 183 Apr 7, 2018 (NY)
1187 183A Apr 21, 2018 (HK)
1188 184 May 16, 2018 (NY)
1190 184A Jun 9, 2018 (HK)
1192 185 Jun 20, 2018 (NY)
1184 182A Mar 24, 2018 (HK)
1196 186 Sep 7, 2018 (NY)
1197 185A Sep 22, 2018 (HK)
1165 175A Sep 23, 2017 (HK)
1168 177 Oct 13, 2017 (NY)
1202 187 Oct 12, 2018 (NY)
1172 178 Nov 18, 2017 (NY)
1173 177A Dec 9, 2017 (HK)
1174 179 Dec 13, 2017 (NY)
1177 181A Jan 27, 2018 (HK)
1

In [168]:
raw.head()

,Auction.Date,Auction.ID,Auction.Name,BottleQuantity,BottleSummary,Conditions,Designation,LotNo,Producer,ProducerAndDesignation,Size,SoldAmount,Vintage,Wine
0,None,1185,"183 Apr 7, 2018 (NY)",1,1 magnum,1x1 magnum owc lscl,Hommage a Jacques Perrin,1,Chateau de Beaucastel,"Hommage a Jacques Perrin, Chateau de Beaucastel",magnum,1860.0,1989,Chateauneuf du Pape
1,None,1185,"183 Apr 7, 2018 (NY)",3,3 bottles,1lscl,Hommage a Jacques Perrin,2,Chateau de Beaucastel,"Hommage a Jacques Perrin, Chateau de Beaucastel",bottle,1736.0,2001,Chateauneuf du Pape
2,None,1185,"183 Apr 7, 2018 (NY)",1,1 magnum,1x1 magnum owc,Hommage a Jacques Perrin,2,Chateau de Beaucastel,"Hommage a Jacques Perrin, Chateau de Beaucastel",magnum,1736.0,2001,Chateauneuf du Pape
3,None,1185,"183 Apr 7, 2018 (NY)",1,1 bottle,3.75cm bc cuc gsvl lscl outstanding color and ...,None,3,Chateau Rayas,Chateau Rayas,bottle,13640.0,1929,Chateauneuf du Pape
4,None,1185,"183 Apr 7, 2018 (NY)",1,1 bottle,7.5cm bc excellent color lscl lstl,None,4,Chateau Rayas,Chateau Rayas,bottle,4712.0,1959,Chateauneuf du Pape


In [156]:
raw.shape

(31962, 14)

In [200]:
# need to group by lot number, append most rows, but the price seems to be static so don't sum that
# for now,  filtering away mixed lots by saying anything that has shared lot numbers shoudl be removed

df = raw.drop_duplicates(subset=['Auction.ID', 'LotNo'], keep=False)
df = df.reset_index()
df.shape

(13394, 15)

In [201]:
df.head()

,index,Auction.Date,Auction.ID,Auction.Name,BottleQuantity,BottleSummary,Conditions,Designation,LotNo,Producer,ProducerAndDesignation,Size,SoldAmount,Vintage,Wine
0,0,None,1185,"183 Apr 7, 2018 (NY)",1,1 magnum,1x1 magnum owc lscl,Hommage a Jacques Perrin,1,Chateau de Beaucastel,"Hommage a Jacques Perrin, Chateau de Beaucastel",magnum,1860.0,1989,Chateauneuf du Pape
1,3,None,1185,"183 Apr 7, 2018 (NY)",1,1 bottle,3.75cm bc cuc gsvl lscl outstanding color and ...,None,3,Chateau Rayas,Chateau Rayas,bottle,13640.0,1929,Chateauneuf du Pape
2,4,None,1185,"183 Apr 7, 2018 (NY)",1,1 bottle,7.5cm bc excellent color lscl lstl,None,4,Chateau Rayas,Chateau Rayas,bottle,4712.0,1959,Chateauneuf du Pape
3,5,None,1185,"183 Apr 7, 2018 (NY)",1,1 bottle,7.5cm bc bsl excellent color,None,5,Chateau Rayas,Chateau Rayas,bottle,3472.0,1961,Chateauneuf du Pape
4,6,None,1185,"183 Apr 7, 2018 (NY)",2,2 bottles,1 excellent color 1 outstanding color and cond...,None,6,Chateau Rayas,Chateau Rayas,bottle,4960.0,1978,Chateauneuf du Pape


In [203]:
# convert to dollars
HKDUSD = 0.1277
def toUSD(row):
    if 'HK' in row['Auction.Name']:
       return row['SoldAmount'] * HKDUSD
    else:
       return row['SoldAmount'] 

df['SoldAmountUSD'] = df.apply( toUSD, axis = 1 )
    
# compute unit price by bottle
df['UnitPriceUSD' = df[ 'SoldAmountUSD' ] / df[ 'BottleQuantity' ]

# bottle sizes only for now
bottle_df = df[ df["BottleSummary"].str.contains( "bottle") ]
bottle_df.shape

(11055, 17)

In [218]:
bottle_grouped = bottle_df.groupby(['Wine', 'Vintage', "Producer"]).UnitPriceUSD.agg( ['min', 'mean', 'max', "count"] )
bottle_grouped = bottle_grouped.reset_index()
bottle_grouped = bottle_grouped.sort_values('count', ascending = False)

In [219]:
mouton_grouped = bottle_grouped[ bottle_grouped["Wine"].str.contains("Mouton") ]

In [220]:
mouton_all = bottle_df[ bottle_df["Wine"].str.contains( "Mouton") ]

In [222]:
mouton_grouped.head()

,Wine,Vintage,Producer,min,mean,max,count
1438,Chateau Mouton Rothschild,2000,Chateau Mouton Rothschild,1766.346400,1999.909783,2365.642500,25
1425,Chateau Mouton Rothschild,1982,Chateau Mouton Rothschild,992.000000,1215.591410,1482.000000,23
1428,Chateau Mouton Rothschild,1986,Chateau Mouton Rothschild,806.000000,905.878661,1033.333333,23
1436,Chateau Mouton Rothschild,1998,Chateau Mouton Rothschild,232.500000,500.521879,633.392000,11
1434,Chateau Mouton Rothschild,1995,Chateau Mouton Rothschild,499.413417,535.796175,568.333333,10


In [224]:
mouton_1982 = mouton_all[ mouton_all['Vintage'] == "1982"]

In [225]:
mouton_1982.head()

,index,Auction.Date,Auction.ID,Auction.Name,BottleQuantity,BottleSummary,Conditions,Designation,LotNo,Producer,ProducerAndDesignation,Size,SoldAmount,Vintage,Wine,SoldAmountUSD,UnitPriceUSD
319,457,None,1185,"183 Apr 7, 2018 (NY)",12,12 bottles,1vhs 1wc 1x12 bottle chateau mouton rothschild...,None,369,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,16120.0,1982,Chateau Mouton Rothschild,16120.000,1343.333333
320,458,None,1185,"183 Apr 7, 2018 (NY)",10,10 bottles,1x12 bottle owc 2bn 2cuc 2spc 4nc 8lbsl,None,370,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,12400.0,1982,Chateau Mouton Rothschild,12400.000,1240.000000
579,793,None,1185,"183 Apr 7, 2018 (NY)",10,10 bottles,1x12 bottle owc 4ts 6bn,None,646,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,12400.0,1982,Chateau Mouton Rothschild,12400.000,1240.000000
1232,707,None,1187,"183A Apr 21, 2018 (HK)",12,12 bottles,12vlscl 1lstl 1vhs 2nc 2ts 8bn 8lwrl,None,593,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,119040.0,1982,Chateau Mouton Rothschild,15201.408,1266.784000
1278,760,None,1187,"183A Apr 21, 2018 (HK)",2,2 bottles,1bn 1lbsl 1lgsl 1ts,None,644,Chateau Mouton Rothschild,Chateau Mouton Rothschild,bottle,19840.0,1982,Chateau Mouton Rothschild,2533.568,1266.784000


In [238]:
plt.hist( mouton_1982[ 'UnitPriceUSD'], 6 )
plt.show()

In [241]:
Vintage = "1988"
Producer = "Leoville Las Cases"
mouton_all = bottle_df[ bottle_df["Wine"].str.contains( Producer) ]
mouton_1982 = mouton_all[ mouton_all['Vintage'] == Vintage ]

plt.hist( mouton_1982[ 'UnitPriceUSD'], 6 )
plt.show()

In [259]:
bottle_profit = bottle_grouped[ bottle_grouped["max"] / 1.25 < bottle_grouped["min"] ]
bottle_profit = bottle_profit[ bottle_profit["count"] > 5 ]
bottle_profit.head()

,Wine,Vintage,Producer,min,mean,max,count
1241,Chateau Latour,2000,Chateau Latour,841.117333,922.503863,1029.166667,19
3934,Screaming Eagle Cabernet Sauvignon,2015,Screaming Eagle,2686.666667,3126.434963,3306.666667,18
2669,Hundred Acre Cabernet Sauvignon,2011,Hundred Acre,226.416667,249.421569,267.583333,17
2853,Les Forts de Latour,2010,chateau de la tour,248.000000,258.608889,263.913333,15
1313,Chateau Leoville Las Cases,1982,Chateau Leoville Las Cases,316.696000,358.716496,392.666667,13


In [260]:
bottle_profit.shape

(77, 7)

In [ ]:
#TODO list!
#PnL 
#Trend over time
#Expected inventory, distributions of profit? strategies? always buy < max?)
#Lot sizes makes difference?
#Ullage level?
#Label condition?